In [2]:
# =========================================================
# 1. Imports
# =========================================================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif

from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt

In [3]:
# =========================================================
# 2. Load dataset
# =========================================================
df = pd.read_csv("data/online_shoppers_intention.csv")
print("Dataset loaded:", df.shape)

Dataset loaded: (12330, 18)


In [4]:
# =========================================================
# 3. Convert boolean variables to numeric
# =========================================================
df['Revenue'] = df['Revenue'].astype(int)
df['Weekend'] = df['Weekend'].astype(int)


# =========================================================
# 4. Group rare categories in numeric-categorical features
# =========================================================
numeric_categorical = ['OperatingSystems', 'Browser', 'Region', 'TrafficType']

def group_rare(series, threshold=50):
    freq = series.value_counts()
    return series.apply(lambda x: x if freq[x] > threshold else "Other")

for col in numeric_categorical:
    df[col] = df[col].astype(str)
    df[col] = group_rare(df[col])


# =========================================================
# 5. Define categorical & numerical sets
# =========================================================
categorical_nominal = ['Month', 'VisitorType'] + numeric_categorical
numerical_features_original = [
    'Administrative', 'Administrative_Duration',
    'Informational', 'Informational_Duration',
    'ProductRelated', 'ProductRelated_Duration',
    'BounceRates', 'ExitRates', 'PageValues',
    'SpecialDay'
]


# =========================================================
# 6. Train-test split (BEFORE encoding!)
# =========================================================
X = df.drop('Revenue', axis=1)
y = df['Revenue']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, " Test:", X_test.shape)


# =========================================================
# 7. Log-transform duration columns
# =========================================================
for col in ['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']:
    X_train[col] = np.log1p(X_train[col])
    X_test[col] = np.log1p(X_test[col])


# =========================================================
# 8. Fit OneHotEncoder on TRAIN split
# =========================================================
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoder.fit(X_train[categorical_nominal])

# Transform
X_train_cat = encoder.transform(X_train[categorical_nominal])
X_test_cat  = encoder.transform(X_test[categorical_nominal])

X_train = X_train.drop(columns=categorical_nominal)
X_test = X_test.drop(columns=categorical_nominal)

# Get feature names (optional)
encoded_cols = encoder.get_feature_names_out(categorical_nominal)


# =========================================================
# Define numerical columns AUTOMATICALLY
# =========================================================
numerical_features = X_train.select_dtypes(include=[np.number]).columns

X_train_num = X_train[numerical_features].reset_index(drop=True)
X_test_num  = X_test[numerical_features].reset_index(drop=True)


X_train_full = np.hstack([X_train_num.values, X_train_cat])
X_test_full  = np.hstack([X_test_num.values,  X_test_cat])



# =========================================================
# 10. Scale numerical + encoded features
# =========================================================
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled  = scaler.transform(X_test_full)


# =========================================================
# 11. PCA (for visualization only)
# =========================================================
pca = PCA(n_components=2)
pca.fit(X_train_scaled)

X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# =========================================================
# 12. SMOTE (APPLY ONLY ON SCALED DATA, ONLY FOR TRAINING MODELS THAT NEED IT)
# =========================================================
# Example: only for tree models, not for LogisticRegression

smote = SMOTE(random_state=0)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print("After SMOTE:", X_train_smote.shape)


Train: (9864, 17)  Test: (2466, 17)
After SMOTE: (16676, 55)


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_pca.py:606: RuntimeWarning: divide by zero encountered in matmul
  C = X.T @ X
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_pca.py:606: RuntimeWarning: overflow encountered in matmul
  C = X.T @ X
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_pca.py:606: RuntimeWarning: invalid value encountered in matmul
  C = X.T @ X
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: divide by zero encountered in matmul
  X_transformed = X @ self.components_.T
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: overflow encountered in matmul
  X_transformed = X @ self.components_.T
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/decomposition/_base.py:148: RuntimeWarning: invalid value encounter

# 1. Linear Discriminant Analysis 

In [ ]:
# =========================================================
# 1. Imports
# =========================================================

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

ImportError: cannot import name 'RegularizedDiscriminantAnalysis' from 'sklearn.discriminant_analysis' (/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py)

In [6]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_scaled, y_train)
y_pred_lda = lda.predict(X_test_scaled)

print("LDA Accuracy:", accuracy_score(y_test, y_pred_lda))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lda))
print("Classification Report:\n", classification_report(y_test, y_pred_lda))

LDA Accuracy: 0.8767234387672344
Confusion Matrix:
 [[2047   37]
 [ 267  115]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.98      0.93      2084
           1       0.76      0.30      0.43       382

    accuracy                           0.88      2466
   macro avg       0.82      0.64      0.68      2466
weighted avg       0.86      0.88      0.85      2466



/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: divide by zero encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: overflow encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: invalid value encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWa

Imabalanced data performs badly with LDA. LDA uses class priors in both training and prediction. If one class is much larger, the model can become biased toward it. Strategies:

1. Adjust Class Priors.

2. Use Resampling Before LDA.

3. Use Regularized LDA (Shrinkage).

## 1.1. Adjust Class Priors

In [7]:
lda_adjusted = LinearDiscriminantAnalysis(priors=[0.5, 0.5])  
lda_adjusted.fit(X_train_scaled, y_train)

y_pred_lda_adjusted = lda_adjusted.predict(X_test_scaled)

print("LDA Accuracy:", accuracy_score(y_test, y_pred_lda_adjusted))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lda_adjusted))
print("Classification Report:\n", classification_report(y_test, y_pred_lda_adjusted))

LDA Accuracy: 0.8824006488240065
Confusion Matrix:
 [[1940  144]
 [ 146  236]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93      2084
           1       0.62      0.62      0.62       382

    accuracy                           0.88      2466
   macro avg       0.78      0.77      0.77      2466
weighted avg       0.88      0.88      0.88      2466



/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: divide by zero encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: overflow encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: invalid value encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWa

## 1.2. Use Resampling before LDA

In [8]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_smote, y_train_smote)
y_pred_lda_smote = lda.predict(X_test_scaled)

print("LDA Accuracy:", accuracy_score(y_test, y_pred_lda_smote))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lda_smote))
print("Classification Report:\n", classification_report(y_test, y_pred_lda_smote))

LDA Accuracy: 0.7846715328467153
Confusion Matrix:
 [[1659  425]
 [ 106  276]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.80      0.86      2084
           1       0.39      0.72      0.51       382

    accuracy                           0.78      2466
   macro avg       0.67      0.76      0.69      2466
weighted avg       0.86      0.78      0.81      2466



/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: divide by zero encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: overflow encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: invalid value encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWa

## 1.3. Use Regularized LDA (Shrinkage)

In [9]:
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
lda.fit(X_train_scaled, y_train)
y_pred_lda = lda.predict(X_test_scaled)

print("LDA Accuracy:", accuracy_score(y_test, y_pred_lda))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lda))
print("Classification Report:\n", classification_report(y_test, y_pred_lda))

LDA Accuracy: 0.8771289537712895
Confusion Matrix:
 [[2048   36]
 [ 267  115]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.98      0.93      2084
           1       0.76      0.30      0.43       382

    accuracy                           0.88      2466
   macro avg       0.82      0.64      0.68      2466
weighted avg       0.87      0.88      0.85      2466



/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


## 1.4. Apply K-Fold Cross Validation to the best result

We have seen that adjust class 

In [35]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
import numpy as np

kf = KFold(n_splits=25, random_state=1, shuffle=True)

cvScores = []
i = 1

for train_index, test_index in kf.split(X_train_scaled):
    print(f"\nFold {i} =============================================================")
    
    X_train_cv, X_val_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_val_cv = y_train.iloc[train_index], y_train.iloc[test_index]
    
    model = LinearDiscriminantAnalysis(priors=[0.5, 0.5]) 
    model.fit(X_train_cv, y_train_cv)
    
    y_pred = model.predict(X_val_cv)
    
    f1 = f1_score(y_val_cv, y_pred, pos_label=1)
    print(f"F1-score (class 1): {f1:.4f}")
    
    cvScores.append(f1)
    i += 1

print("\nMean F1:", np.mean(cvScores))
print("Std F1:", np.std(cvScores))



Fold 1 =============================================================
F1-score (class 1): 0.5586

Fold 2 =============================================================
F1-score (class 1): 0.5524

Fold 3 =============================================================
F1-score (class 1): 0.6538

Fold 4 =============================================================
F1-score (class 1): 0.6281

Fold 5 =============================================================
F1-score (class 1): 0.6154

Fold 6 =============================================================
F1-score (class 1): 0.6939

Fold 7 =============================================================
F1-score (class 1): 0.6379

Fold 8 =============================================================
F1-score (class 1): 0.5785

Fold 9 =============================================================


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: divide by zero encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: overflow encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: invalid value encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWa

F1-score (class 1): 0.6349

Fold 10 =============================================================
F1-score (class 1): 0.7132

Fold 11 =============================================================
F1-score (class 1): 0.5692

Fold 12 =============================================================
F1-score (class 1): 0.5909

Fold 13 =============================================================
F1-score (class 1): 0.6142

Fold 14 =============================================================
F1-score (class 1): 0.6483

Fold 15 =============================================================
F1-score (class 1): 0.5577

Fold 16 =============================================================
F1-score (class 1): 0.5739

Fold 17 =============================================================
F1-score (class 1): 0.6126

Fold 18 =============================================================
F1-score (class 1): 0.6500

Fold 19 =============================================================
F1-score (class 1): 

/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: divide by zero encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: overflow encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: invalid value encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWa

F1-score (class 1): 0.6777

Fold 22 =============================================================
F1-score (class 1): 0.6667

Fold 23 =============================================================
F1-score (class 1): 0.6613

Fold 24 =============================================================
F1-score (class 1): 0.7132

Fold 25 =============================================================
F1-score (class 1): 0.6306

Mean F1: 0.6260940666399495
Std F1: 0.046117043133240394


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: divide by zero encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: overflow encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: invalid value encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWa

# 2. Quadratical Discriminant Analysis

In [10]:
# Initialize and train the QDA model
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train_scaled, y_train)

# Make predictions
y_pred_qda = qda.predict(X_test_scaled)

# Evaluate the model
print("QDA Accuracy:", accuracy_score(y_test, y_pred_qda))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_qda))
print("Classification Report:\n", classification_report(y_test, y_pred_qda))

QDA Accuracy: 0.6232765612327656
Confusion Matrix:
 [[1234  850]
 [  79  303]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.59      0.73      2084
           1       0.26      0.79      0.39       382

    accuracy                           0.62      2466
   macro avg       0.60      0.69      0.56      2466
weighted avg       0.83      0.62      0.68      2466



QDA is more sensitive to imbalanced data than LDA. The covariance matriz is computed from very few samples, so it gets unstable and QDA estimates one covariance matrix per class. 
The prior heavily favors the majority class.

We are going to apply the same methods to deal with imbalanced data.

## 2.1. Adjust Class Priors

In [11]:
qda_adjusted = QuadraticDiscriminantAnalysis(priors=[0.5, 0.5])  
qda_adjusted.fit(X_train_scaled, y_train)

y_pred_qda_adjusted = qda_adjusted.predict(X_test_scaled)

print("QDA Accuracy:", accuracy_score(y_test, y_pred_qda_adjusted))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_qda_adjusted))
print("Classification Report:\n", classification_report(y_test, y_pred_qda_adjusted))

QDA Accuracy: 0.551094890510949
Confusion Matrix:
 [[1046 1038]
 [  69  313]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.50      0.65      2084
           1       0.23      0.82      0.36       382

    accuracy                           0.55      2466
   macro avg       0.58      0.66      0.51      2466
weighted avg       0.83      0.55      0.61      2466



## 2.2. Use Resampling before QDA

In [12]:
qda = LinearDiscriminantAnalysis()
qda.fit(X_train_smote, y_train_smote)
y_pred_qda_smote = lda.predict(X_test_scaled)

print("LDA Accuracy:", accuracy_score(y_test, y_pred_qda_smote))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_qda_smote))
print("Classification Report:\n", classification_report(y_test, y_pred_qda_smote))

LDA Accuracy: 0.8771289537712895
Confusion Matrix:
 [[2048   36]
 [ 267  115]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.98      0.93      2084
           1       0.76      0.30      0.43       382

    accuracy                           0.88      2466
   macro avg       0.82      0.64      0.68      2466
weighted avg       0.87      0.88      0.85      2466



/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: divide by zero encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: overflow encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/discriminant_analysis.py:629: RuntimeWarning: invalid value encountered in matmul
  self.scalings_ = scalings @ Vt.T[:, :rank]
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWa

## 2.3. Use Regularized QDA

In [20]:
qda_regularized = QuadraticDiscriminantAnalysis(reg_param=0.9)  
qda_regularized.fit(X_train_scaled, y_train)

y_pred_qda_regularized = qda_regularized.predict(X_test_scaled)

print("QDA Accuracy:", accuracy_score(y_test, y_pred_qda_regularized))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_qda_regularized))
print("Classification Report:\n", classification_report(y_test, y_pred_qda_regularized))

QDA Accuracy: 0.8349553933495539
Confusion Matrix:
 [[1867  217]
 [ 190  192]]
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.90      0.90      2084
           1       0.47      0.50      0.49       382

    accuracy                           0.83      2466
   macro avg       0.69      0.70      0.69      2466
weighted avg       0.84      0.83      0.84      2466



# 3. Regularized Discriminant Analysis (RDA)

In [22]:
import numpy as np

class RegularizedDiscriminantAnalysis:
    """
    Implementation of Regularized Discriminant Analysis (RDA)
    as described in Friedman (1989).
    
    Parameters
    ----------
    lambda_param : float, default=0.0
        Controls blending between class-specific covariance (0)
        and pooled covariance (1).

    gamma_param : float, default=0.0
        Controls shrinkage of covariance toward spherical matrix.

    Attributes
    ----------
    classes_ : array-like
        Unique class labels.
    priors_ : array-like
        Class prior probabilities.
    cov_ : dict
        Regularized covariance matrix per class.
    means_ : dict
        Class means.
    """

    def __init__(self, lambda_param=0.0, gamma_param=0.0):
        self.lambda_param = lambda_param
        self.gamma_param = gamma_param

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        n_samples, n_features = X.shape

        # Compute class means, covariances
        self.means_ = {}
        covariances = {}
        priors = {}

        for c in self.classes_:
            X_c = X[y == c]
            priors[c] = X_c.shape[0] / n_samples
            self.means_[c] = X_c.mean(axis=0)
            
            # Sample covariance for that class
            covariances[c] = np.cov(X_c, rowvar=False)

        self.priors_ = priors

        # Compute pooled covariance
        pooled_cov = np.zeros((n_features, n_features))
        for c in self.classes_:
            n_c = (y == c).sum()
            pooled_cov += (n_c - 1) * covariances[c]
        pooled_cov /= (n_samples - n_classes)

        # Build regularized covariances (Friedman's RDA)
        self.cov_ = {}

        for c in self.classes_:
            cov_k = covariances[c]

            # Step 1: blend class-specific vs pooled
            cov_lambda = (
                (1 - self.lambda_param) * cov_k
                + self.lambda_param * pooled_cov
            )

            # Step 2: shrink toward spherical covariance
            trace_val = np.trace(cov_lambda) / n_features
            cov_gamma = (
                (1 - self.gamma_param) * cov_lambda
                + self.gamma_param * trace_val * np.eye(n_features)
            )

            self.cov_[c] = cov_gamma

        return self

    def _discriminant(self, X, mean, cov, prior):
        """Compute discriminant function for Gaussian model."""
        inv_cov = np.linalg.inv(cov)
        det_cov = np.linalg.det(cov)
        diff = X - mean

        # Quadratic discriminant function
        term1 = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
        term2 = -0.5 * np.log(det_cov)
        term3 = np.log(prior)

        return term1 + term2 + term3

    def predict(self, X):
        X = np.asarray(X)
        scores = []

        for c in self.classes_:
            s = self._discriminant(
                X,
                self.means_[c],
                self.cov_[c],
                self.priors_[c]
            )
            scores.append(s)

        scores = np.vstack(scores).T
        idx = np.argmax(scores, axis=1)
        return self.classes_[idx]

    def predict_proba(self, X):
        X = np.asarray(X)
        scores = []

        for c in self.classes_:
            s = self._discriminant(
                X,
                self.means_[c],
                self.cov_[c],
                self.priors_[c]
            )
            scores.append(s)

        scores = np.vstack(scores).T
        
        # Convert log-scores to probabilities (softmax)
        exp_scores = np.exp(scores - scores.max(axis=1, keepdims=True))
        return exp_scores / exp_scores.sum(axis=1, keepdims=True)


In [31]:
p_lambda = []
p_gamma = []
f1_score = []

for lam in np.arange(0,1.1,0.1):
    for gam in np.arange(0,1.1,0.1):
        # Initialize and train the QDA model
        rda = RegularizedDiscriminantAnalysis(lambda_param=lam,gamma_param=gam)
        rda.fit(X_train_scaled, y_train)

        # Make predictions
        y_pred_rda = rda.predict(X_test_scaled)

        report = classification_report(y_test, y_pred_rda, output_dict=True)
        f1_class_1 = report["1"]["f1-score"]
        print(f1_class_1)

        p_lambda.append(lam)
        p_gamma.append(gam)
        f1_score.append(f1_class_1)

idx_f1_score = f1_score.index(max(f1_score))
max_f1_score = max(f1_score)
max_p_lambda = p_lambda[idx_f1_score]
max_p_gamma = p_gamma[idx_f1_score]

print("Max f1-score: ", max_f1_score)
print("Lambda: ", max_p_lambda)
print("Gamma: ", max_p_gamma)

# # Evaluate the model
# print("QDA Accuracy:", accuracy_score(y_test, y_pred_rda))
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rda))
# print("Classification Report:\n", classification_report(y_test, y_pred_rda))

# report = classification_report(y_test, y_pred_rda, output_dict=True)
# f1_class_1 = report["1"]["f1-score"]


/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/var/folders/gx/8_x0j9wx7kl8gs36bydwh7l00000gr/T/ipykernel_2523/328652228.py:94: RuntimeWarning: divide by zero encountered in matmul
  term1 = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
/var/folders/gx/8_x0j9wx7kl8gs36bydwh7l00000gr/T/ipykernel_2523/328652228.py:94: RuntimeWarning: overflow encountered in matmul
  term1 = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
/var/folders/gx/8_x0j9wx7kl8gs36bydwh7l00000gr/T/ipykernel_2523/3

0.3947882736156352
0.42593957258658804
0.4307944307944308
0.4469964664310954
0.4533333333333333
0.4579533941236069
0.4466230936819172
0.4431687715269805
0.44657863145258103
0.45838668373879643
0.46648793565683644
0.41025641025641024
0.4356120826709062
0.4442508710801394
0.45209302325581396
0.45714285714285713
0.4505844845908608
0.4471635150166852
0.4462616822429907
0.45145631067961167
0.4623376623376623
0.46630727762803237
0.42298850574712643
0.4434931506849315
0.4546287809349221
0.4564796905222437
0.46138002059732236
0.45751633986928103
0.4497716894977169
0.4489795918367347
0.46326276463262767
0.45931758530183725
0.46236559139784944
0.43140495867768597
0.45248868778280543
0.45542635658914726
0.4613821138211382
0.4588744588744589
0.45598194130925507
0.45176470588235296
0.4579780755176614
0.4672657252888318
0.46622516556291393
0.4609164420485175
0.4421052631578947
0.4584139264990329
0.46279306829765543
0.46702702702702703
0.4617117117117117
0.45901639344262296
0.4552058111380145
0.46326

/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/var/folders/gx/8_x0j9wx7kl8gs36bydwh7l00000gr/T/ipykernel_2523/328652228.py:94: RuntimeWarning: divide by zero encountered in matmul
  term1 = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
/var/folders/gx/8_x0j9wx7kl8gs36bydwh7l00000gr/T/ipykernel_2523/328652228.py:94: RuntimeWarning: overflow encountered in matmul
  term1 = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
/var/folders/gx/8_x0j9wx7kl8gs36bydwh7l00000gr/T/ipykernel_2523/3

0.46618106139438087
0.4647577092511013
0.46490218642117376
0.4595238095238095
0.4621026894865526
0.45942571785268416
0.4675324675324675
0.4715447154471545
0.46408839779005523
0.4685408299866131
0.47132429614181437
0.47404063205417607
0.47238542890716806
0.4682926829268293
0.46192259675405745
0.46114649681528663
0.47493403693931396
0.46831955922865015
0.47009735744089015
0.4728789986091794
0.4738955823293173
0.4699074074074074
0.4638036809815951
0.45685279187817257
0.4589308996088657
0.46442953020134226
0.4712328767123288
0.4774011299435028
0.4733044733044733
0.47467438494934877
0.4767277856135402
0.4772117962466488
0.4751958224543081
0.4738292011019284
0.47875354107648727
0.4797687861271676
0.4815361890694239
0.47058823529411764
0.47806354009077157
0.4854961832061069
0.4835820895522388
0.4857142857142857
0.4817320703653586
0.48297213622291024
0.48796147672552165
0.4828711256117455
0.48344370860927155
0.4774624373956594
0.4709784411276949
0.4752475247524752
0.4935897435897436
0.49230769

/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/tania_priv/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/var/folders/gx/8_x0j9wx7kl8gs36bydwh7l00000gr/T/ipykernel_2523/328652228.py:94: RuntimeWarning: divide by zero encountered in matmul
  term1 = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
/var/folders/gx/8_x0j9wx7kl8gs36bydwh7l00000gr/T/ipykernel_2523/328652228.py:94: RuntimeWarning: overflow encountered in matmul
  term1 = -0.5 * np.sum(diff @ inv_cov * diff, axis=1)
/var/folders/gx/8_x0j9wx7kl8gs36bydwh7l00000gr/T/ipykernel_2523/3